# Proyecto 2. Entrega 2: Árboles de Decisión
### Integrantes
- Nelson García  
- Diego Linares
- Joaquin Puente
- José Mérida
- Joaquín Campos

Las secciones de análisis exploratorio y separación del modelo se realizaron en la entrega anterior, sin embargo consideramos importante tener esta información a la mano para poder referenciarla dentro de este mismo documento.

## Análisis exploratorio

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn.tree import DecisionTreeRegressor, plot_tree


### Carga de Datos y Revisión General

In [ ]:
# estaremos definiendo ambos csvs para poder tener acceso a ambos pero usaremos el train y luego sobre el test replicaremos una vez se considere importante

# Definir NA como nuestros NaN
dftrain = pd.read_csv("train.csv", na_values=["NA"])

# incluimos en el analissi exploratorio lo basico para poder tenerlo a la mano
dftrain.head()

In [ ]:
# datos estadísticos básicos
dftrain.describe()

In [ ]:
# tipos
dftrain.info()

In [ ]:
# Revisamos posibles variables redundantes entre sí mismas
print(dftrain.nunique())

### Separación de Columnas por Tipo de Variable
Algunas columnas cómo MSSubClass son categóricas nominales y cuentan con valores numéricos, además algunas otras columnas cómo pueden ser las de clasificación de estado son ordinales y tienen un encoding categórico (ej. Ex - Excelente en ExterCond). Es importante que clasifiquemos las diferentes variables para poder llevar a cabo el encoding de manera correcta y realizar nuestro análisis exploratorio. Tenemos 3 categorías:

- Ordinales
- Nominales
- Numéricas

Este proceso se lleva a cabo antes de depurar las columnas que no se utilizarán en el procesamiento de datos para evitar la necesidad de mantener en mente las columnas eliminadas al categorizar.

In [ ]:
col_ordinales = ['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                 'BsmtExposure', 'HeatingQC',  'GarageQual', 'GarageCond', 'FireplaceQu',  'Functional',
                 'KitchenQual', 'PoolQC', 'Fence']
col_nominales = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
                 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
                 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical',
                 'GarageFinish','GarageType','PavedDrive', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition']
col_numericas = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
                 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
                 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
                 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']

Definimos una función para retornar el tipo de variable de cada columna

In [ ]:
def get_type(col):
  if col in col_numericas + ['SalePrice']:
    return 'numerica'
  elif col in col_nominales:
    return 'nominal'
  elif col in col_ordinales:
    return 'ordinal'

Verificamos que todas las columnas se hayan ingresado correctamente

In [ ]:
unassigned = []
for col in dftrain.columns:
  if get_type(col) == None:
    unassigned.append(col)

print(unassigned)

### Datos Faltantes
En este paso vamos a analizar datos faltantes

Primero verificamos las columnas con valores nulos, para tener una mejor idea de que necesitamos hacer

In [ ]:
missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Nos damos cuenta que PoolQC tiene 99.5% de valores faltantes, ¿Por qué?

Algunas de las variables categóricas tienen cómo categoría "NA" y se toma como valor nulo al cargar los datos al DF. Creamos una lista con las variables que cuentan con esta característica

In [ ]:
na_as_data_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                   'FireplaceQu', 'GarageType', 'GarageFinish', 'PoolQC', 'Fence', 'MiscFeature',
                   'MasVnrType', 'GarageQual', 'GarageCond']

dftrain[na_as_data_cols] = dftrain[na_as_data_cols].fillna('Missing')

Revisando nuevamente las columnas con valores nulos

In [ ]:
missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Ahora tenemos valores faltantes más manejables, primero vamos a tomar LotFrontage y GarageYrBlt y reemplazar los valores faltantes con la mediana. Esto debido a que el 17.73% y 5.54% siguen siendo cifras bastante significativas y no podemos simplemente eliminar estas filas.

In [ ]:
cols_to_fill = ['LotFrontage', 'GarageYrBlt']

medians = dftrain[cols_to_fill].median()

dftrain[cols_to_fill] = dftrain[cols_to_fill].fillna(medians)

Revisando nuevamente los valores nulos

In [ ]:
missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Ahora con una pequeña cantidad de valores faltantes, podemos simplemente remover las filas que los contengan.

In [ ]:
dftrain = dftrain.dropna()

missing_values = dftrain.isnull().sum() / len(dftrain) * 100
print(missing_values.sort_values(ascending=False))

Ya no tenemos valores faltantes dentro de nuestro DF

### Encoding de Variables Categóricas
En este paso vamos a codificar las variables categóricas, las ordinales utilizando OrdinalEncoder y las nominales utilizando get_dummies. De esta manera podemos utilizar las variables ordinales cómo numéricas y aplicar las nominales a nuestro análisis.

Identificamos los diferentes valores que puedan tomar las variables dentro del dataset

In [ ]:
for col in col_ordinales:
    print(f"{col}: {dftrain[col].unique()}")

Aplicamos el encoding

In [ ]:
# Diferentes categorías para las diferentes columnas
standard_categories = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'Missing']
fence_categories = ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'Missing']
bsmt_exposure_categories = ['No', 'Mn', 'Av', 'Gd', 'Missing']
functional_categories = ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Missing']

cleaned_ordinales = [x for x in col_ordinales if x not in ['OverallQual', 'OverallCond']]

# Limpiamos los entries quitando posibles errores de espacios
dftrain[cleaned_ordinales] = dftrain[cleaned_ordinales].astype(str).apply(lambda x: x.str.strip())

# Asignación de categorías a columnas
categories = []
for col in cleaned_ordinales:
    if col == 'Fence':
        categories.append(fence_categories)
    elif col == 'BsmtExposure':
        categories.append(bsmt_exposure_categories)
    elif col == 'Functional':
        categories.append(functional_categories)
    else:
        categories.append(standard_categories)

# Inicialización y aplicación de encoder
encoder = OrdinalEncoder(categories=categories, handle_unknown='use_encoded_value', unknown_value=-1)
dftrain[cleaned_ordinales] = encoder.fit_transform(dftrain[cleaned_ordinales]).astype(int)

# Verificacion de filas sin encodear
print((dftrain[cleaned_ordinales] == -1).sum())

# Conteo de valores por cada columna
for col in cleaned_ordinales:
    print(f"{col}:")
    print("\n".join([f"{val} - {count}" for val, count in dftrain[col].value_counts().items()]))
    print()


Para las variables nominales, utilizamos get_dummies.

In [ ]:
dftrain = pd.get_dummies(dftrain, columns=col_nominales, prefix_sep='_')
dftrain

Con el output podemos observar que ya se encuentran codificadas nuestras variables categóricas ordinales utilizando valores numéricos.

### Depuración de Datos
En este paso revisamos si existen datos duplicados

Eliminación de Filas Duplicadas

In [ ]:
before = dftrain.shape[0]

# Eliminar duplicados
dftrain = dftrain.drop_duplicates()
after = dftrain.shape[0]
print(f"Filas eliminadas: {before - after}")

No existen filas duplicadas.

dftrain.drop('id', axis=2, inplace=True)

### Exploración Variable de Respuesta
En este paso buscamos obtener más información sobre la variable respuesta, ya que nuestro interés es buscar predecirla.

¿Cómo se distribuye?

In [ ]:
sns.histplot(dftrain['SalePrice'], kde=True, color='skyblue', bins=30).set(title='Distribucion de SalePrice', xlabel='Precio de Venta', ylabel='Cantidad de Casas')

La variable SalePrice sigue una distribución cerca de la normal, con un sesgo hacia la derecha. Esto quiere decir que hay más casas con precios bajos a medios y pocas con precios muy altos. A parte del análisis gráfico, podemos obtener algunos datos adicionales

In [ ]:
print(dftrain['SalePrice'].describe())
print("Skewness:", dftrain['SalePrice'].skew())

¿Cuáles variables se correlacionan con la variable objetivo?

In [ ]:
# cuántas categorías únicas hay por columna?
print(dftrain.select_dtypes(include=['object']).nunique())

print("Distribucion de categorias por columna: ")
# Distribución de categorías por columna
for col in dftrain.select_dtypes(include=['object']).columns:
    print(dftrain[col].value_counts())

Gracias a este output podemos "observar" de forma rápida, que algunas de las variables como utilities, un poco LandSlope, Condition2 y tal vez otras variables pueden ser eliminadas, pero necesitamos poder justificar, de esta forma igual ya nos podemos hacer una idea de como hay algunas variables que tienen poca relevancia.

### Exploración Variables Categóricas Ordinales
En este paso buscamos obtener más información sobre las variables categóricas ordinales, buscando identificar cómo se distribuyen y que nos dicen sobre las casas del dataset

In [ ]:
# Generar gráficos para cada variable ordinal con respecto a SalePrice
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(18, 18))
axes = axes.flatten()

for i, col in enumerate(col_ordinales):
    if col in dftrain.columns and dftrain[col].nunique() > 1:
        sns.boxplot(data=dftrain, x=col, y="SalePrice", ax=axes[i])
        axes[i].set_title(f"Distribución de SalePrice según {col}")
        axes[i].set_xlabel(col)
        axes[i].set_ylabel("SalePrice")
    else:
        axes[i].axis('off')  # Ocultar gráficos vacíos

plt.tight_layout()
plt.show()


Con base en la información que nos presentan los gráficos anteriores

Podemos concluir que
- La calidad general (OverallQual) es el factor ordinal más importante para determinar SalePrice.
- Las características internas como KitchenQual y FireplaceQu tienen un fuerte impacto en el valor de la vivienda.
- Condiciones estructurales (OverallCond, Functional) tienen menos influencia.
- Elementos adicionales como piscinas pueden elevar significativamente el precio, pero su presencia es rara.

### Exploración Variables Categóricas Nominales

In [ ]:
# Seleccionar solo las columnas codificadas
encoded_cols = [col for col in dftrain.columns if any(col.startswith(nom) for nom in col_nominales)]

# Calcular la correlación de las variables categóricas nominales con SalePrice
correlations = dftrain[encoded_cols + ['SalePrice']].corr()['SalePrice'].sort_values(ascending=False)

# Graficar las correlaciones más significativas
correlations = correlations.dropna()
correlations = correlations[correlations.abs() > 0.1]  # Filtrar solo las más relevantes

plt.figure(figsize=(12, 6))
sns.barplot(x=correlations.index, y=correlations.values, palette='coolwarm')
plt.xticks(rotation=90)
plt.xlabel("Variables Categóricas Nominales Codificadas")
plt.ylabel("Correlación con SalePrice")
plt.title("Correlación entre Variables Categóricas Nominales y SalePrice")
plt.show()


Con base en la gráfica anterior, podemos observar que

- Ubicación (Neighborhood) es uno de los factores más determinantes en el precio de venta, se menciona por todos los campos que incluyen a (Neighborhood)
- La calidad del cimiento (Foundation), el sótano (BsmtFinType1), el acabado del garaje (GarageFinish) y el tipo de vivienda también influyen significativamente.
- Ciertas configuraciones como garajes separados o sistemas de calefacción deficientes pueden reducir el valor.
- Algunas variables nominales, aunque intuitivamente importantes, tienen una baja correlación y pueden no ser determinantes.

### Exploracion Variables Numéricas

In [ ]:
df_num = dftrain.select_dtypes(include = ['float64', 'int64'])
df_num = df_num.drop('Id', axis=1)
# Visualizacion y observacion variables numericas
df_num.hist(figsize=(20, 18), bins=30)
plt.show()

Gracias a esto podemos imaginar que hay bastantes variables con nulas, o como wooddeckSF en donde parece que la varianza es baja podemos identificar tambien que ademas hay bastantes varaibles que pueden sernos de gran utilidad en la busqueda de salesPrice pero sobre todo variables que tienden a una moda y como estas distribuciones que podriamos usar.

In [ ]:
# Crear un solo gráfico con subplots
num_cols = len(df_num.columns)
ncols = 3
nrows = (num_cols // ncols) + (num_cols % ncols > 0)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows * 3))
axes = axes.flatten()

for i, col in enumerate(df_num.columns):
    sns.boxplot(x=df_num[col], ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xlabel("")

# Ocultar gráficos vacíos si hay menos columnas que subplots
total_axes = len(axes)
for i in range(num_cols, total_axes):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

Segun esto podemos ver que la correlacion entre las variables númericas pero aún asi no podemos descartar ninguna variable porque la correlacion no tienen numeros altos para poder eliminarla con justificacion.

## Separación del modelo

Gracias a ambos analisis, podemos determinar que la variable con mas relacion a SalePrice, es OverallQual, de esta forma para hacer la particion del subconjunto de prueba y entrenamiento, podemos ayudarnos de esta.

Se usará la particion 80/20 de manera que el 80% del dataset(train.csv) se usará para entrenamiento, y el 20% del dataset(train.csv) se usara para prueba.

In [ ]:
train_set_strat, test_set_strat = train_test_split(
    dftrain,
    test_size=0.2,
    stratify=dftrain["OverallQual"],
    random_state=42  #Asegura reproducibilidad
)

print("Tamaño de train:", train_set_strat.shape[0])
print("Tamaño de test:", test_set_strat.shape[0])

train_distribution = train_set_strat["OverallQual"].value_counts(normalize=True).sort_index()
test_distribution = test_set_strat["OverallQual"].value_counts(normalize=True).sort_index()

distribution_df = pd.DataFrame({
    "OverallQual": train_distribution.index,
    "Train Proportion": train_distribution,
    "Test Proportion": test_distribution.reindex(train_distribution.index, fill_value=0)
}).reset_index(drop=True)

print(distribution_df)

## Árbol de Regresión (Sin Max Depth)
En esta sección implementamos un árbol de regresión para predecir el precio de las casas utilizando todas las variables. Se utiliza para predecir el precio de las casas y se analiza el rendimiento

### Entrenamiento del Arbol
En esta sección separamos los splits X e Y de entrenamiento y prueba, utilizando el DecisionTreeRegressor de SKLearn para entrenar nuestro modelo

Separación de splits

In [ ]:
X_train = train_set_strat.drop(columns=['SalePrice'])
Y_train = train_set_strat['SalePrice']

X_test = test_set_strat.drop(columns=['SalePrice'])
Y_test = test_set_strat['SalePrice']

Entrenamiento del árbol de regresión

In [ ]:
regressor = DecisionTreeRegressor(random_state = 42)

regressor.fit(X_train, Y_train)

### Predicciones
En esta sección aplicamos nuestra regresión a ambos conjuntos, el conjunto de prueba y el conjunto de entrenamiento. Esto con la finalidad de determinar el rendimiento de nuestro modelo y poder determinar si existe overfitting al utilizar todas las variables disponibles.

In [ ]:
Y_pred = regressor.predict(X_test)
Y_train_pred = regressor.predict(X_train)

### Análisis
En esta sección indagamos sobre los indicadores y gráficas necesarias para determinar y explicar el rendimiento de nuestro modelo.

***Análisis MSE, MAE y R²***

In [ ]:
mse = mean_squared_error(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Test")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

mse = mean_squared_error(Y_train, Y_train_pred)
mae = mean_absolute_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

print("Train")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

**Rendimiento en Entrenamiento**
- R²: 1.0
- MSE: 0.0
- MAE: 0.0

**Rendimiento en Prueba**
- R²: 0.71
- MSE: 1906093576.18
- MAE: 27635.80


_R² de 1.0 en el set de entrenamiento, ¿El modelo perfecto?_

No, el rendimiento en el entrenamiento sugiere un performance completamente perfecto. Esto lo que indica, es que el modelo se adaptó **demasiado** bien al conjunto de entrenamiento y se memorizó todo el ruido y los datos atípicos presentes dentro de este split. Lo que se busca dentro del modelo es que se encuentren patrones generalizables para poder predecir correctamente otros sets de datos.

_Rendimiento de Prueba_

Al analizar las estadísticas en el conjunto de prueba, nos damos cuenta que efectivamente si hay overfitting. La diferencia en rendimiento es bastante significativa, por lo que la adapción tan cercana al conjunto de entrenamiento prueba ser contraproducente.

_Posibles Causas_

Es bastante común que los árboles de regresión sufran de overfitting. Además, por default el DecisionTreeRegressor en scikit-learn no limita la profundidad del árbol. Eso aumenta el overfitting, ya que este árbol seguirá creciendo hasta que llegue a ajustarse completamente a los datos de entrenamiento.

_Ajustes_

Una manera que se pueden explorar ajustes en el modelo es modificar el parámetro de profundidad. Esto previene que el árbol siga creciendo indefinidamente sobre-ajustándose a los datos de prueba y busque predecir de manera perfecta cada dato atípico o particularidad de los datos.


***Información Sobre el Árbol***

Debido a la profundidad del árbol, resulta sumamente complicado graficarlo y llegar a conclusiones basados en la gráfica. Debido a esto, decidimos obtener diferentes características del árbol y analizarlo de esa manera

In [ ]:
print(f"Profundidad: {regressor.tree_.max_depth}")
print(f"Numero de Hojas: {regressor.get_n_leaves()}")
print(f"Samples Promedio por Hoja: {np.mean(regressor.tree_.n_node_samples[np.where(regressor.tree_.children_left == regressor.tree_.children_right)[0]]):.2f}")
print(f"Hoja con Más Samples: {np.max(regressor.tree_.n_node_samples[np.where(regressor.tree_.children_left == regressor.tree_.children_right)[0]])}")
print(f"Cantidad de Datos de Entrenamiento:  {len(train_set_strat)}")

Los datos obtenidos respaldan nuestro análisis anterior, explicando la existencia del over-fitting. ¿Por qué?

*Profundidad: 26*

Esto nos indica que hay 26 niveles diferentes dónde se separan los datos, lo cuál es una profundidad bastante alta para el árbol.

*Número de Hojas: 1112*

Estos son los nodos dónde "terminan" los datos para predecir su precio, teniendo 1160 datos y 1112 hojas indica que existe un nodo específico para cada uno de los datos casi todas las veces. Esto nos lleva a cumplir condiciones **sumamente específicas** para llegar a cada uno, adaptándose muy bien a los datos de entrenamiento pero talvez no a los de prueba.

*Samples Promedio por Hoja: 1.04*

Respaldando el análisis del inciso anterior, la cantidad de nodos por hoja es sumamente baja. Esto quiere decir que el árbol esta prediciendo específicamente cada uno de los datos, en vez de buscar patrones más generalizables que puedan categorizar datos nuevos.

*Hoja con Más Samples: 3*

El 'grupo' más grande dentro del grupo de entrenamiento del árbol fue de 3 samples, el modelo no logró reconocer alguna otra similaridad entre los datos

*Conclusiones*

La profundidad y la forma del árbol explica el overfitting descrito en el análisis de eficiencia del modelo. Además, apoya la sugerencia de variar el parámetro de profundidad para llegar a un modelo que tenga la capacidad de reconocer patrones generales y no específicos del set de entrenamiento.

***Gráficas***

**Valores Reales vs Predicciones**

Buscamos analizar de manera gráfica los valores reales en comparación de los predecidos por el modelo

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Valores Reales vs Predicciones")
plt.show()

En la gráfica podemos observar lo siguiente:

- Los puntos parecen estar distribuidos bastante uniformemente entre estar por encima y por abajo de la línea roja. Esto indica que no hay un over / under prediction significativo

- Los valores pequeños se encuentran bastante cercanos a la línea, sin embargo mientras esta crece se van alejando más. Indicando que los errores crecen para valores más altos. Esto puede darse debido a la alta diferencia entre valores (100,000 vs 600,000) o errores dentro del modelo.

- Existen algunos valores atípicos bastante alejados de los demás puntos, esto se puede dar debido a la inhabilidad del modelo de encontrar algunos patrones más complejos. O bien, debido al over-fitting.

In [ ]:
residuals = Y_test - Y_pred

plt.figure(figsize=(8, 6))
plt.scatter(Y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Horizontal line at 0
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

La gráfica de residuos nos indica que la precisión del modelo decrece conforme el valor de la propiedad aumenta. Esta información se puede utilizar para variar los parámetros del árbol, o transformar variables de manera que se ajusten de mejor manera a la regresión.

In [ ]:
# Get feature importances
feature_importances = regressor.feature_importances_

# Create a DataFrame to store feature names and their importances
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance (descending order)
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the top 10 most important features
print("Top 10 Most Important Features:")
print(importance_df.head(10))

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'][:10], importance_df['Importance'][:10], color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Most Important Features')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important feature at the top
plt.show()

## Árbol de Regresión (Max Depth = 13)
El objetivo principal de esta iteración es evitar el over-fitting, por lo que decidimos cortar la profundidad a la mitad. Esperamos ver un modelo que se adapte mejor a patrones generales, en vez de intentar agrupar cada uno de los datos de entrenamiento dentro de su misma categoría.

### Entrenamiento del Modelo y Predicciones

In [ ]:
X_train = train_set_strat.drop(columns=['SalePrice'])
Y_train = train_set_strat['SalePrice']

X_test = test_set_strat.drop(columns=['SalePrice'])
Y_test = test_set_strat['SalePrice']

regressor = DecisionTreeRegressor(random_state = 42, max_depth = 13)

regressor.fit(X_train, Y_train)

Y_pred = regressor.predict(X_test)
Y_train_pred = regressor.predict(X_train)

### Rendimiento del Modelo

In [ ]:
mse = mean_squared_error(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Test")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

mse = mean_squared_error(Y_train, Y_train_pred)
mae = mean_absolute_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

print("Train")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

### Gráficas

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Valores Reales vs Predicciones")
plt.show()

In [ ]:
residuals = Y_test - Y_pred

plt.figure(figsize=(8, 6))
plt.scatter(Y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Horizontal line at 0
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

## Árbol de Regresión (Max Depth = 4)


### Entrenamiento del Modelo y Predicciones

In [ ]:
X_train = train_set_strat.drop(columns=['SalePrice'])
Y_train = train_set_strat['SalePrice']

X_test = test_set_strat.drop(columns=['SalePrice'])
Y_test = test_set_strat['SalePrice']

regressor = DecisionTreeRegressor(random_state = 42, max_depth =4)

regressor.fit(X_train, Y_train)

Y_pred = regressor.predict(X_test)
Y_train_pred = regressor.predict(X_train)

### Rendimiento del modelo

In [ ]:
mse = mean_squared_error(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Test")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

mse = mean_squared_error(Y_train, Y_train_pred)
mae = mean_absolute_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

print("Train")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

### Gráficas

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Valores Reales vs Predicciones")
plt.show()

In [ ]:
residuals = Y_test - Y_pred

plt.figure(figsize=(8, 6))
plt.scatter(Y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Horizontal line at 0
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

### Árbol de regresión (Max Depth = X)


## Fine-Tuning
El objetivo en esta sección es diseñar un método similar al método del "codo" aplicado en clustering.

*¿Por qué?*

- Conforme aumenta el depth, la precisión del modelo en el conjunto de prueba empieza a llegar a un punto fijo.

- Conforme aumenta el depth, aumenta el overfitting. Dónde se continúa adaptando al conjunto de entrenamiento.

Lo que se busca es encontrar el punto dentro del parámetro de profundidad dónde se deja de ajustar a las características de los datos y empieza a memorizarlos.

*¿Cómo?*

Nuestra propuesta de solución es hacer una gráfica de R2 de prueba y R2 de entrenamiento en comparación al parámetro de max_depth. El objetivo es buscar el punto dónde el R2 de prueba empiece a "aplanarse", similar al codo. Además, podemos validar la efectividad de nuestro método planteado al verificar que el R2 de entrenamiento siga aumentando.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

depths = range(1, 21)

train_r2 = []
test_r2 = []

for depth in depths:
    regressor = DecisionTreeRegressor(max_depth=depth, random_state=42)
    regressor.fit(X_train, Y_train)

    Y_train_pred = regressor.predict(X_train)
    train_r2.append(r2_score(Y_train, Y_train_pred))

    Y_test_pred = regressor.predict(X_test)
    test_r2.append(r2_score(Y_test, Y_test_pred))

plt.figure(figsize=(10, 6))
plt.plot(depths, train_r2, marker='o', linestyle='--', label='Training R²')
plt.plot(depths, test_r2, marker='o', linestyle='--', label='Test R²')
plt.xlabel('Tree Depth')
plt.ylabel('R²')
plt.title('Training and Test R² vs Tree Depth')
plt.legend()
plt.grid(True)

selected_point = 7
plt.axvline(x=selected_point, color='red', linestyle='--', label=f'Punto de Interés (Depth = {selected_point})')
plt.legend()

plt.show()

*Análisis de la Gráfica*

La gráfica generada es bastante interesante, podemos observar algunos de los siguientes puntos clave:

- Como se predijo, la curva de Training R2 se va aplanando acercándose cada vez más a 1.

- La curva de R2 de entrenamiento parece seguir un trend similar, sin embargo, en contra de las predicciones en lugar de "aplanarse" empieza a tener un comportamiento errático.

*¿Por qué?*

Basados en el análisis realizado anteriormente, sabemos que un mayor ajuste a los datos de entrenamiento puede tener un impacto tanto positivo cómo negativo. Podemos intuir que luego de este punto, el modelo empieza a sobre-ajustar al conjunto de prueba. Luego, la oscilación del modelo puede explicarse cómo un indicador de qué tantas características "pegaron" a los datos de entrenamiento que por "casualidad" fueron seleccionadas en alguna profundidad. Cuándo se empieza a acercar a 1 el R2 de entrenamiento, esto nos indica que únicamente se están dividiendo de manera diferente (basado en diferentes características) los datos y realmente el ajuste con el set de prueba no es un indicador real de la calidad del modelo.

*Solución*

Decidimos utilizar un R2 "cross-validated" para evitar ligeramente la influencia de "casualidades" entre ambos datasets.

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = []
for depth in depths:
    regressor = DecisionTreeRegressor(max_depth=depth, random_state=42)
    scores = cross_val_score(regressor, X_train, Y_train, cv=5, scoring='r2')
    cv_scores.append(np.mean(scores))

plt.figure(figsize=(10, 6))
plt.plot(depths, cv_scores, marker='o', linestyle='--', label='Cross-Validated R²')
plt.xlabel('Tree Depth')
plt.ylabel('Cross-Validated R²')
plt.title('Cross-Validated R² vs Tree Depth')
plt.legend()
plt.grid(True)
plt.show()

*Análisis de la Gráfica*

En esta gráfica podemos observar que las oscilaciones cambian ligeramente en comparación a la anterior. Esto nos indica que nuestra predicción fue correcta, las oscilaciones simplemente eran causadas por la diferencia en importancia de características en cada  modelo.

*Conclusión*

Debido al comportamiento errático del rendimiento del modelo cuándo las predicciones del conjunto de prueba comienzan a ser perfectas, decidimos buscar minimizar el parámetro de depth para nuestro modelo. Además, buscamos el punto dónde encontremos un mayor valor de R2 con las predicciones del modelo del conjunto de prueba. Por eso, seleccionamos cómo número "óptimo" max_depth = 7.

- Se encuentra antes del comportamiento errático
- Su valor de R2 es similar en el calculado con los conjuntos y cross-validated
- Se maximiza el performance del modelo en comparación a puntos anteriores 

## Árbol de Regresión (Max Depth = 7)

### Entrenamiento del Modelo y Predicciones

In [ ]:
X_train = train_set_strat.drop(columns=['SalePrice'])
Y_train = train_set_strat['SalePrice']

X_test = test_set_strat.drop(columns=['SalePrice'])
Y_test = test_set_strat['SalePrice']

regressor = DecisionTreeRegressor(random_state = 42, max_depth = 7)

regressor.fit(X_train, Y_train)

Y_pred = regressor.predict(X_test)
Y_train_pred = regressor.predict(X_train)

### Rendimiento del modelo

In [ ]:
mse = mean_squared_error(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Test")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

mse = mean_squared_error(Y_train, Y_train_pred)
mae = mean_absolute_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

print("Train")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

### Gráficas

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Y_test, Y_pred, alpha=0.5)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Valores Reales vs Predicciones")
plt.show()

In [ ]:
residuals = Y_test - Y_pred

plt.figure(figsize=(8, 6))
plt.scatter(Y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')  # Horizontal line at 0
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

## Comparación con Regresión Lineal (5)

## Creación Variable de Respuesta

## Análisis de Eficiencia

## Validación Cruzada